# TRY MORE WINDOWS
# IF peaks collide, take the bigger one (remove these peaks altogther?)


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
import sys
import logging

tf.get_logger().setLevel(logging.ERROR)

sys.path.append('../src/')

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

np.set_printoptions(suppress=True)

In [ ]:
# import modules
from cnn.simulator import Simulator

from cnn.models import ConvNet
from cnn.losses import CustomLoss
from cnn.metrics import CustomAUC
from cnn.preprocessing import LabelEncoder
from cnn.data_generators import DataGenerator

In [ ]:
simulator_train = Simulator(LabelEncoder(512))

fig, axes = plt.subplots(36, 1, figsize=(20, 120))
for i in range(36):
    x = simulator_train._generate_example(i+100)['chromatogram']
    axes.reshape(-1)[i].plot(x)
    axes.reshape(-1)[i].set_xlim(5000, 7500)
    axes.reshape(-1)[i].set_ylim(-2, 2)

In [ ]:

# Model training
NUM_TRAIN_EXAMPLES = int(1e5)
NUM_TEST_EXAMPLES = int(1e4)
BATCH_SIZE = 32
STEPS_PER_EPOCH = 1e5 // BATCH_SIZE
NUM_EPOCHS = 2

# Model optimizer
INITIAL_LEARNING_RATE = 1e-3
END_LEARNING_RATE = 1e-5
DECAY_STEPS = int(STEPS_PER_EPOCH / BATCH_SIZE * NUM_EPOCHS)

# Label encoder
NUM_CLASSES = 3
NUM_WINDOWS = 512

# Define loss function, optimizer and metrics
loss_fn = CustomLoss(NUM_CLASSES, 5.0, 1.0, 1.0)
optimizer = tf.keras.optimizers.Adam(
    learning_rate=tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=INITIAL_LEARNING_RATE, 
        decay_steps=DECAY_STEPS, 
        end_learning_rate=END_LEARNING_RATE
    )
)

metrics = [CustomAUC(NUM_CLASSES)]

# Define data generators 
# Simulator uses method of label_encoder to deal with collisions
label_encoder = LabelEncoder(NUM_WINDOWS)
train_generator = DataGenerator(
    np.arange(0, NUM_TRAIN_EXAMPLES), 
    simulator=Simulator(label_encoder), 
    label_encoder=label_encoder,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_generator = DataGenerator(
    np.arange(NUM_TRAIN_EXAMPLES, NUM_TRAIN_EXAMPLES + NUM_TEST_EXAMPLES), 
    simulator=Simulator(label_encoder), 
    label_encoder=label_encoder,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

model = ConvNet(
    filters=[128, 128, 256, 256, 512],
    kernel_sizes=[9, 9, 9, 9, 9],
    pool_type='max',
    input_shape=(16384, 1),
    output_shape=(NUM_WINDOWS, NUM_CLASSES)
)

model.compile(loss=loss_fn, optimizer=optimizer, metrics=metrics)

In [ ]:
model.fit(
    train_generator, validation_data=test_generator,
    epochs=NUM_EPOCHS, steps_per_epoch=STEPS_PER_EPOCH,
)

In [ ]:
prediction = model.predict(test_generator, verbose=1)

In [ ]:
# for i in range(313):
#     true = test_generator.__getitem__(i)
#     if true[1][0][:, 0].sum() < 24:
#         print(i)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 10))

x_lim = [0.0, 1.0 + 1e-7]

ID = 6

true_move = {0: (0.0, 0), 1: (0, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0), 
             5: (0, 0), 6: (0, 0), 7: (0, 0), 8: (0, 0), 9: (0, 0), 
             10: (0, 0), }

pred_move = {0: (0, 0), 1: (0, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0), 
             5: (0, 0), 6: (0.03, 0), 7: (-0.005, 15), 8: (0, 0), 9: (0.03, 0), 
             10: (0, 15), }


# GROUND TRUTH
true = test_generator.__getitem__(ID)

pred = prediction[ID*test_generator.batch_size]

probs, locs, areas = label_encoder.decode(pred, 0.4)

x = true[0][0] * 100
y = true[1][0]
#x = x * 255

gt = label_encoder.decode(y)

j, k = 0, 0
for i, y_sample in enumerate(y):
    if y_sample[0] == 1:
        
        index = int(gt[1][k] * 16384)
        height = max(x[index:index+1]) + 5
        
        if x_lim[0] < gt[1][k] < x_lim[1]: 
            ax.scatter(gt[1][k], np.min(x)-10, c='C0', s=75, marker='^', edgecolor='black')
            #ax.axvline(gt[1][k], linestyle='--')
            #if k % 3 == 0:
            ax.text(x=gt[1][k]-0.005 * (x_lim[1] - x_lim[0]), y=height+12, s=f"{gt[2][k]:.3f}", color="C0", fontsize=20)
        
        k += 1
        
    if pred[i][0] > 0.4:
        index = int(gt[1][k-1] * 16384)
        height = max(x[index:index+1]) + 5
        
        if x_lim[0] < locs[j] < x_lim[1]: 
            
         
            ax.scatter(locs[j], np.min(x)-60, c='C1', s=75, marker='^', edgecolor='black')
            ax.text(x=locs[j]-0.005 * (x_lim[1] - x_lim[0]), y=np.min(x)-155, s=f'{probs[j]:.3f}', color='black', rotation=-45, fontsize=12)
            ax.text(x=locs[j]-0.005 * (x_lim[1] - x_lim[0]), y=height-30, s=f"{areas[j]:.3f}", color='C1', fontsize=20)

        j += 1
        

        
ax.plot(np.linspace(0, 1, 16384), x, c='black')
        
ax.scatter([None],[None], c='C0', label='Ground truth')
ax.scatter([None],[None], c='C1', label='Predicted')
ax.plot([None],[None], c='C0', label=' ')
ax.plot([None],[None], c='C1', label=' ')


ax.set_ylabel('Absorbance', fontsize=20)
ax.set_xlabel('Time', fontsize=20)
ax.set_ylim(np.min(x)-200, np.max(x)+200);
ax.set_xlim(*x_lim)
ax.tick_params(axis='both', length=5, labelsize=20)

start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 0.1))
start, end = ax.get_ylim()
#ax.yaxis.set_ticks(np.arange(0, end, 50))
ax.legend(columnspacing=-0.5, fontsize=20, ncol=2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


#plt.savefig('peak_detection_2.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 10))

x_lim = [0.2, 0.6 + 1e-7]

ID = 6

true_move = {0: (0.0, 0), 1: (0, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0), 
             5: (0, 0), 6: (0, 0), 7: (0, 0), 8: (0, 0), 9: (0, 0), 
             10: (0, 0), }

pred_move = {0: (0, 0), 1: (0, 0), 2: (0, 0), 3: (0, 0), 4: (0, 0), 
             5: (0, 0), 6: (0.03, 0), 7: (-0.005, 15), 8: (0, 0), 9: (0.03, 0), 
             10: (0, 15), }


# GROUND TRUTH
true = test_generator.__getitem__(ID)

pred = prediction[ID*test_generator.batch_size]

probs, locs, areas = label_encoder.decode(pred, 0.4)

x = true[0][0] * 100
y = true[1][0]
#x = x * 255

gt = label_encoder.decode(y)

j, k = 0, 0
for i, y_sample in enumerate(y):
    if y_sample[0] == 1:
        
        index = int(gt[1][k] * 16384)
        height = max(x[index:index+1]) + 5
        
        if x_lim[0] < gt[1][k] < x_lim[1]: 
            ax.scatter(gt[1][k], np.min(x)-10, c='C0', s=75, marker='^', edgecolor='black')
            #ax.axvline(gt[1][k], linestyle='--')
            #if k % 3 == 0:
            ax.text(x=gt[1][k]-0.005 * (x_lim[1] - x_lim[0]), y=height+42, s=f"{gt[2][k]:.3f}", color="C0", fontsize=20)
        
        k += 1
        
    if pred[i][0] > 0.4:
        index = int(gt[1][k-1] * 16384)
        height = max(x[index:index+1]) + 5
        
        if x_lim[0] < locs[j] < x_lim[1]: 
            
         
            ax.scatter(locs[j], np.min(x)-60, c='C1', s=75, marker='^', edgecolor='black')
            ax.text(x=locs[j]-0.005 * (x_lim[1] - x_lim[0]), y=np.min(x)-155, s=f'{probs[j]:.3f}', color='black', rotation=-45, fontsize=12)
            ax.text(x=locs[j]-0.005 * (x_lim[1] - x_lim[0]), y=height, s=f"{areas[j]:.3f}", color='C1', fontsize=20)

        j += 1
        

        
ax.plot(np.linspace(0, 1, 16384), x, c='black')
        
ax.scatter([None],[None], c='C0', label='Ground truth')
ax.scatter([None],[None], c='C1', label='Predicted')
ax.plot([None],[None], c='C0', label=' ')
ax.plot([None],[None], c='C1', label=' ')


ax.set_ylabel('Absorbance', fontsize=20)
ax.set_xlabel('Time', fontsize=20)
ax.set_ylim(np.min(x)-200, np.max(x)+200);
ax.set_xlim(*x_lim)
ax.tick_params(axis='both', length=5, labelsize=20)

start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 0.1))
start, end = ax.get_ylim()
#ax.yaxis.set_ticks(np.arange(0, end, 50))
ax.legend(columnspacing=-0.5, fontsize=20, ncol=2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

#ax.set_ylim(0, 20)
#plt.savefig('peak_detection_2.png', dpi=300)

In [ ]:
import scipy.interpolate
import glob
import pandas as pd

In [ ]:
def apply_interpolation(x, y, target_size):
    '''
    Resizes a chromatogram to a given size/resolution
    '''
    f = scipy.interpolate.interp1d(x, y)
    xnew = np.linspace(x.min(), x.max(), target_size)
    ynew = f(xnew)
    return xnew, ynew

def apply_interpolation(y, target_size):
    '''
    Resizes a chromatogram to a given size/resolution
    '''
    x = np.linspace(0, 1, len(y))
    f = scipy.interpolate.interp1d(x, y)
    xnew = np.linspace(0, 1, target_size)
    ynew = f(xnew)
    return ynew

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(20, 10))

csv = pd.read_csv('../input/chrom_1/ISO_ALL_F160.CSV', header=None, sep='\t', encoding='utf-16')
x = csv[1].values
x = apply_interpolation(x, 16384)
x_max = x.max()
x = x[None, :, None] / x_max
out = model.predict(x)[0]
x *= 100

for c in glob.glob('../input/chrom_1/ISO_[0-9]*'):
    csv = pd.read_csv(c, header=None, sep='\t', encoding='utf-16')
    d = csv[1].values
    d = apply_interpolation(d, 16384) / x_max * 100
    idx = np.where(d > 0.05)[0]
    s, e = idx[0], idx[-1]
    ex = 50 + int((100 * np.linspace(0, 1, 16384)[s]))
    axes[1].plot(np.linspace(0, 1, 16384)[s-ex:e+ex], d[s-ex:e+ex], c='C0', linewidth=3)
    
    
probs, locs, areas = label_encoder.decode(out, 0.5)
for i, (p, l, a) in enumerate(zip(probs, locs, areas)):
    axes[1].scatter(l, np.min(x)-40, marker='^', c=f'C1')
    axes[1].text(x=l, y=-70, s=f'{p:.3f}', rotation=-45)
    axes[1].axvline(l, c='C1', linestyle=':', linewidth=2)
    axes[0].axvline(l, c='C1', linestyle=':', linewidth=2)
    
axes[0].plot(np.linspace(0, 1, 16384), x[0, :, 0], c='black', linestyle='-')

axes[0].set_xlim(0.1, 0.5)
axes[1].set_xlim(0.1, 0.5)

axes[0].tick_params(
    axis='x', which='both', bottom=False, top=False, labelbottom=False)
axes[1].spines['top'].set_visible(False)
#axes[1].axhline(1.05, c='black', linewidth=1, linestyle=':')

ymin, ymax = axes[0].get_ylim()
axes[0].set_ylim(ymin-65, ymax+5)
axes[1].set_ylim(ymin-65, ymax+5)
plt.subplots_adjust(hspace=-0.2)
#axes[0].set_ylim(-5, 5)

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(20, 10))

chromatogram = np.zeros(16384)

for c in glob.glob('../input/ISO10/*.CSV'):
    csv = pd.read_csv(c, header=None, sep='\t', encoding='utf-16')
    d = csv[1].values
    d = apply_interpolation(d, 16384)
    chromatogram += d
    axes[0].plot(np.linspace(0, 1, 16384), d)
    
axes[1].plot(np.linspace(0, 1, 16384), chromatogram)
axes[1].set_xlim(0.0, 0.8)
axes[0].set_xlim(0.0, 0.8)


x = apply_interpolation(chromatogram, 16384)
x_max = x.max()
x = x[None, :, None] / x_max
out = model.predict(x)[0]
x *= x_max

probs, locs, areas = label_encoder.decode(out, 0.6)
for i, (p, l, a) in enumerate(zip(probs, locs, areas)):
    axes[1].scatter(l, np.min(x)-40, marker='^', c=f'C1')
    #axes[1].text(x=l, y=-70, s=f'{p:.3f}', rotation=-45)
    #axes[0].axvline(l, c='C1', linestyle=':', linewidth=2)
    #axes[0].axvline(l, c='C1', linestyle=':', linewidth=2)
    

plt.tight_layout()
#plt.savefig('ISO_10_all_compounds.png', dpi=200)

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(20, 10))

chromatogram = np.zeros(16384)

for c in glob.glob('../input/ISO40/*.CSV'):
    csv = pd.read_csv(c, header=None, sep='\t', encoding='utf-16')
    d = csv[1].values
    d = apply_interpolation(d, 16384)
    chromatogram += d
    axes[0].plot(np.linspace(0, 1, 16384), d)
    
axes[1].plot(np.linspace(0, 1, 16384), chromatogram)
axes[1].set_xlim(0.0, 0.8)
axes[0].set_xlim(0.0, 0.8)


x = apply_interpolation(chromatogram, 16384)
x_max = x.max()
x = x[None, :, None] / x_max
out = model.predict(x)[0]
x *= x_max

probs, locs, areas = label_encoder.decode(out, 0.4)
for i, (p, l, a) in enumerate(zip(probs, locs, areas)):
    axes[1].scatter(l, np.min(x)-40, marker='^', c=f'C1')
    #axes[1].text(x=l, y=-70, s=f'{p:.3f}', rotation=-45)
    #axes[0].axvline(l, c='C1', linestyle=':', linewidth=2)
    #axes[0].axvline(l, c='C1', linestyle=':', linewidth=2)
    

plt.tight_layout()

#plt.ylim(-5, 5)
#plt.savefig('ISO_10_all_compounds.png', dpi=200)

In [ ]:
sorted_index = np.array([ 0, 21, 20,  6, 22, 18, 48,  5, 10,  7, 12, 49, 31, 13,  8,  9, 35,
       33, 11, 23, 25, 38, 37, 47, 41, 46, 42,  3, 39, 45,  1,  4, 16, 26,
       24, 17, 14, 34, 29,  2, 19, 40, 28, 44, 30, 27, 32, 36, 15, 43])

fig, axes = plt.subplots(2,1, figsize=(20, 10))

chromatogram = np.zeros(16384)

files = np.array(glob.glob('../input/ISO30/*.CSV'))[sorted_index]
compounds = [f.split('/')[-1].split('.CSV')[0] for f in files]

keep_list = [
    '27_ISO30',    'XVII_ISO30',     'XLVIII_ISO30', '73_ISO30',    
    'XXXV_ISO30',  'LXI_ISO30',    '77_ISO30',      'LVI_ISO30',  
    'XV_ISO30',    'XXXVIII_ISO30',  '05_ISO30',
    'XXIII_ISO30', '68_ISO30',      'XLIII_ISO30',
    'XXVII_ISO30', '78_ISO30',     '39_ISO30',      '67_ISO30',     'XL_ISO30',
    'LV_ISO30',       '38_ISO30',      'XI_ISO30',     '66_ISO30',
    'LIV_ISO30',   'X_ISO30',        '76_ISO30',     '24_ISO30',
    'XXIX_ISO30',  'XII_ISO30',        '59_ISO30',     'XLI_ISO30',
    'XXXIII_ISO30', '02_ISO30',      '81_ISO30',     '17_ISO30',
    'XLVI_ISO30'
]

rt = []
for file, compound in zip(files, compounds):
    
    if compound in keep_list:
        
        csv = pd.read_csv(file, header=None, sep='\t', encoding='utf-16')
        peak = csv[1].values
        peak = apply_interpolation(peak, 16384)
        
        index = np.argmax(peak)
        rt.append(index)
        chromatogram += peak
        axes[0].plot(np.linspace(0, 1, 16384), peak, label=compound)
        
axes[1].plot(np.linspace(0, 1, 16384), chromatogram)
axes[1].set_xlim(0.2, 0.9)
axes[0].set_xlim(0.2, 0.9)
axes[0].legend(ncol=10)


x = apply_interpolation(chromatogram, 16384)
x_max = x.max()
x = x[None, :, None] / x_max
out = model.predict(x)[0]
x *= x_max

probs, locs, areas = label_encoder.decode(out, 0.4)
for i, (p, l, a) in enumerate(zip(probs, locs, areas)):
    axes[1].scatter(l, np.min(x)-40, marker='^', c=f'C1')
    #axes[1].text(x=l, y=-70, s=f'{p:.3f}', rotation=-45)
    #axes[0].axvline(l, c='C1', linestyle=':', linewidth=2)
    #axes[0].axvline(l, c='C1', linestyle=':', linewidth=2)


plt.tight_layout()




In [ ]:
# import scipy.signal

# plt.figure(figsize=(80, 8))
# deriv = scipy.signal.savgol_filter(x, window_length=51, polyorder=2, deriv=2)

# plt.plot(x[2000:7000] * 0.001)
# plt.plot(deriv[2000:7000])

In [ ]:
# plt.figure(figsize=(60, 10))


# csv = pd.read_csv('../input/chrom_1/GRA_ALL_F160.CSV', header=None, sep='\t', encoding='utf-16')
# x = csv[1].values
# x = apply_interpolation(x, 16384)
# x_max = x.max()
# x = x[None, :, None] / x_max
# out = model.predict(x)[0]

# for c in glob.glob('../input/chrom_1/GRA_[0-9]*'):
#     csv = pd.read_csv(c, header=None, sep='\t', encoding='utf-16')
#     d = csv[1].values
#     d = apply_interpolation(d, 16384)
#     d = d / x_max
#     plt.plot(d)
    
    
# probs, locs, areas = label_encoder.decode(out, 0.5)
# for i, (p, l, a) in enumerate(zip(probs, locs, areas)):
#    # print(p, l, a)
#     plt.scatter(l * 16384, np.min(x)-0.2, c=f'C{i}', label=f'{p:.2f}')
#     #plt.text(x=l * 16384, y=np.min(x)-0, s=f'{p:.2f}', rotation=-45)
#     #plt.plot([l-w/2, l+w/2], [.080+(i*.015), .080+(i*.015)], color=f'C{i}')
#     #plt.plot([l, l], [0, a], color='C0', linewidth=4, linestyle=':')
#     #plt.plot([l-s/2, l+s/2], [0, 0], color='C0', linewidth=4, linestyle=':')
    
#     #index = int(g * 16384)
#     #height = max(x[index-10:index+10]) + 20
#     #plt.text(x=g-0.005, y=height, s=f"{a:.2f}", color='C1', fontsize=14)

# plt.legend(loc=2, ncol=10)
# plt.plot(x[0, :, 0], c='black', linestyle='--')
